In [1]:
!pip --q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Audio
import torch
import numpy as np
import pandas as pd
import soundfile as sf
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def DownloadDataset(username,key):
  import json
  keys={"username":username,"key":key}
  ! mkdir ~/.kaggle
  json_object = json.dumps(keys, indent=4)
  with open(r"/root/.kaggle/kaggle.json", "w") as outfile:
      outfile.write(json_object)
  ! chmod 600 ~/.kaggle/kaggle.json
  ! kaggle datasets download  --unzip engmahmoodanaam/cleansada-version-01

#.............................................

DownloadDataset( username = "engmahmoodanaam",
                 key      = "4a457b4fc3516f9d9b913d770ea64884"
                )

100% 1.03G/1.03G [00:05<00:00, 240MB/s]
100% 1.03G/1.03G [00:05<00:00, 215MB/s]


In [5]:
def GetDataset(path_csv,path_audio):
  df = pd.read_csv(path_csv).sample(frac=1,random_state=42).reset_index(drop=True)
  audios_data = []
  audios_samplerate = []
  for idx, row in df.iterrows():
    filename = f"{path_audio}/{row['SegmentID']}.wav"
    audiodata, samplerate = sf.read(filename)
    audios_data.append(audiodata)
    audios_samplerate.append(samplerate)

  df['audio'] = audios_data
  df['samplerate'] = audios_samplerate
  df['text'] = df['ProcessedText']
  df = df[['text','audio','samplerate']]


  return df

#.............................................


In [6]:
df = GetDataset(
                path_csv   = r'/content/cleanSada-01/data.csv',
                path_audio =  r'/content/cleanSada-01/Audios'
                ).iloc[:70]

df.head()

,text,audio,samplerate
0,هو يعتقد الان بيدخل عشان يشارك في المسابقة فان...,"[0.080841064453125, 0.065155029296875, 0.04803...",16000
1,انقطع الخط اسمعني,"[-0.00244140625, -0.00030517578125, 0.00027465...",16000
2,من داخل المملكة على الرقم اربعة اربعة اثنين خم...,"[-0.076385498046875, -0.090362548828125, -0.11...",16000
3,اؤمر اؤمر يا كبير وانا هنفذ,"[-0.002655029296875, -0.001800537109375, -0.00...",16000
4,انت الحبيبة وهي ام عيالي وماحبها,"[-0.004547119140625, -0.02398681640625, -0.048...",16000


In [7]:
print('df shape: ',df.shape)

df shape:  (70, 3)




---



In [8]:
full_generation_index = 0
full_generation_df = df.iloc[full_generation_index].to_frame().T.reset_index(drop=True)


train_df = df.sample(frac=0.8,random_state=42)
eval_df = df.drop(train_df.index)



print('train_df shape: ',train_df.shape)
print('eval_df shape: ',eval_df.shape)
print('full_generation_df shape: ',full_generation_df.shape)

train_df shape:  (56, 3)
eval_df shape:  (14, 3)
full_generation_df shape:  (1, 3)


In [9]:
print(full_generation_df['text'][0])
Audio(full_generation_df['audio'][0], rate=full_generation_df['samplerate'][0])

هو يعتقد الان بيدخل عشان يشارك في المسابقة فانت تقول انه كان متردد حتى ما ذاكر ولا شيء ولا وش كان يقول


In [10]:
from datasets import DatasetDict,Dataset

ds = {
        "train": Dataset.from_pandas(train_df),
        "eval": Dataset.from_pandas(eval_df),
        "full_generation": Dataset.from_pandas(full_generation_df),
    }

dataset = DatasetDict(ds)

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'audio', 'samplerate', '__index_level_0__'],
        num_rows: 56
    })
    eval: Dataset({
        features: ['text', 'audio', 'samplerate', '__index_level_0__'],
        num_rows: 14
    })
    full_generation: Dataset({
        features: ['text', 'audio', 'samplerate'],
        num_rows: 1
    })
})

In [11]:
dataset.save_to_disk('./dataset')

Saving the dataset (0/1 shards):   0%|          | 0/56 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]



---



In [12]:
!git clone https://github.com/Mahmood-Anaam/VitsModelSplit.git

Cloning into 'VitsModelSplit'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 122 (delta 70), reused 95 (delta 46), pack-reused 0
Receiving objects: 100% (122/122), 20.41 MiB | 28.16 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [13]:
from transformers import AutoTokenizer
from VitsModelSplit.dataset_features_collector import  run_dataset_features_collection
from VitsModelSplit.vits_model import VitsModel
from VitsModelSplit.feature_extraction import VitsFeatureExtractor


In [14]:
model = VitsModel.from_pretrained("facebook/mms-tts-ara",cache_dir="./")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-ara",cache_dir="./")
feature_extractor = VitsFeatureExtractor()

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-tts-ara were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [15]:
run_dataset_features_collection(
                dataset_dir = r'/content/dataset',
                train_split_name='train',
                eval_split_name='eval',
                full_generation_name = 'full_generation',
                tokenizer=tokenizer,
                model=model,
                feature_extractor=feature_extractor,
                train_batch_size=10,
                eval_batch_size=10,
                output_dir= '/content/drive/MyDrive/FeaturesCollectionDataset'
                )

Parameter 'function'=<bound method DataSetFeaturesCollector.prepare_dataset of DataSetFeaturesCollector()> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


preprocess:   0%|          | 0/10 [00:00<?, ? examples/s]

Train Dataset - batch 0, waveform torch.Size([10, 291199, 1]),tokens torch.Size([10, 231])... 


preprocess:   0%|          | 0/10 [00:00<?, ? examples/s]

Train Dataset - batch 1, waveform torch.Size([10, 198079, 1]),tokens torch.Size([10, 271])... 


...

Eval Dataset - batch 0, waveform torch.Size([10, 366400, 1]),tokens torch.Size([10, 513])... 


preprocess:   0%|          | 0/4 [00:00<?, ? examples/s]

Eval Dataset - batch 1, waveform torch.Size([4, 247680, 1]),tokens torch.Size([4, 325])... 


preprocess:   0%|          | 0/1 [00:00<?, ? examples/s]

Full Generation Dataset - batch 0, waveform torch.Size([1, 106880, 1]),tokens torch.Size([1, 205])... 




---

